In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.autograd import Variable
import time

In [2]:
train_dataset = datasets.MNIST(root='./data',
                               train=True,
                              transform=transforms.ToTensor(),
                              download=True)
test_dataset = datasets.MNIST(root='./data',
                               train=False,
                              transform=transforms.ToTensor(),
                              download=True)

In [3]:
batch_size = 100
epochs = 5
iterations = epochs * len(train_dataset)/batch_size
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size,
                                          shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size,
                                          shuffle=False)

In [24]:
class CNNModule(nn.Module):
    def __init__(self):
        super(CNNModule, self).__init__()
        
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=2)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2)
        
        self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2)
        
        self.fc = nn.Linear(32*7*7, 10)
        
    def forward(self, x):
        out = self.cnn1(x)
        out = self.relu(out)
        out = self.maxpool(out)
        
        out = self.cnn2(out)
        out = self.relu(out)
        out = self.maxpool(out)
        
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        
        return out

In [27]:
model = CNNModule()
model.cuda()

criterion = nn.CrossEntropyLoss()
learning_rate = 0.01
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

print(len(list(model.parameters())))

accuracy = -999
itern = 0
start_time = time.time()
for epoch in range(epochs):
    for i, (images, labels) in enumerate(train_loader):
        
        images = Variable(images.cuda())
        labels = Variable(labels.cuda())
        
        optimizer.zero_grad()
        
        outputs = model(images)
        
        loss = criterion(outputs, labels)
        loss_val = loss.data.cpu().numpy().reshape(1)[0]
        loss.backward()
        
        optimizer.step()
        
        itern += 1
        if itern%500 ==0:
            correct = 0
            total = 0
            for test_images, test_labels in test_loader:
                test_images = Variable(test_images.cuda())
                test_preds = model(test_images)
                _, predicted = torch.max(test_preds.data, 1)  
                predicted = predicted.cpu()
                total +=  test_labels.size(0)
                correct += (predicted == test_labels).sum()
            correct = correct.numpy().reshape(1)[0]
            accuracy = 100 * correct/total    
            print('Epoch {}, Iter {}, Loss {}, Accuracy {}'.format(epoch, itern,loss_val, accuracy))
print('Training time {}'.format(time.time() - start_time))

6
Epoch 0, Iter 500, Loss 0.11595166474580765, Accuracy 98.41
Epoch 1, Iter 1000, Loss 0.050977710634469986, Accuracy 98.63
Epoch 2, Iter 1500, Loss 0.0494062565267086, Accuracy 98.7
Epoch 3, Iter 2000, Loss 0.04427538067102432, Accuracy 98.75
Epoch 4, Iter 2500, Loss 0.0006665897089987993, Accuracy 98.54
Epoch 4, Iter 3000, Loss 0.005369591526687145, Accuracy 98.48
Training time 50.494030475616455


In [29]:
class CNNModule_avgpool(nn.Module):
    def __init__(self):
        super(CNNModule_avgpool, self).__init__()
        
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=2)
        self.relu = nn.ReLU()
        self.avgpool = nn.AvgPool2d(kernel_size=2)
        
        self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2)
        
        self.fc = nn.Linear(32*7*7, 10)
        
    def forward(self, x):
        out = self.cnn1(x)
        out = self.relu(out)
        out = self.avgpool(out)
        
        out = self.cnn2(out)
        out = self.relu(out)
        out = self.avgpool(out)
        
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        
        return out

In [31]:
model = CNNModule_avgpool()
model.cuda()

criterion = nn.CrossEntropyLoss()
learning_rate = 0.01
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

print(len(list(model.parameters())))

accuracy = -999
itern = 0
start_time = time.time()
for epoch in range(epochs):
    for i, (images, labels) in enumerate(train_loader):
        
        images = Variable(images.cuda())
        labels = Variable(labels.cuda())
        
        optimizer.zero_grad()
        
        outputs = model(images)
        
        loss = criterion(outputs, labels)
        loss_val = loss.data.cpu().numpy().reshape(1)[0]
        loss.backward()
        
        optimizer.step()
        
        itern += 1
        if itern%500 ==0:
            correct = 0
            total = 0
            for test_images, test_labels in test_loader:
                test_images = Variable(test_images.cuda())
                test_preds = model(test_images)
                _, predicted = torch.max(test_preds.data, 1)  
                predicted = predicted.cpu()
                total +=  test_labels.size(0)
                correct += (predicted == test_labels).sum()
            correct = correct.numpy().reshape(1)[0]
            accuracy = 100 * correct/total    
            print('Epoch {}, Iter {}, Loss {}, Accuracy {}'.format(epoch, itern,loss_val, accuracy))
print('Training time {}'.format(time.time() - start_time))

6
Epoch 0, Iter 500, Loss 0.06245262920856476, Accuracy 98.23
Epoch 1, Iter 1000, Loss 0.012223894707858562, Accuracy 98.67
Epoch 2, Iter 1500, Loss 0.06922449916601181, Accuracy 98.94
Epoch 3, Iter 2000, Loss 0.00919282529503107, Accuracy 98.95
Epoch 4, Iter 2500, Loss 0.01480302307754755, Accuracy 98.72
Epoch 4, Iter 3000, Loss 0.03684893250465393, Accuracy 98.99
Training time 56.35944890975952


In [34]:
class CNNModule_zeropadding(nn.Module):
    def __init__(self):
        super(CNNModule_zeropadding, self).__init__()
        
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=0)
        self.relu = nn.ReLU()
        self.avgpool = nn.AvgPool2d(kernel_size=2)
        
        self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=0)
        
        self.fc = nn.Linear(32*4*4, 10)
        
    def forward(self, x):
        out = self.cnn1(x)
        out = self.relu(out)
        out = self.avgpool(out)
        
        out = self.cnn2(out)
        out = self.relu(out)
        out = self.avgpool(out)
        
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        
        return out

In [35]:
model = CNNModule_zeropadding()
model.cuda()

criterion = nn.CrossEntropyLoss()
learning_rate = 0.01
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

print(len(list(model.parameters())))

accuracy = -999
itern = 0
start_time = time.time()
for epoch in range(epochs):
    for i, (images, labels) in enumerate(train_loader):
        
        images = Variable(images.cuda())
        labels = Variable(labels.cuda())
        
        optimizer.zero_grad()
        
        outputs = model(images)
        
        loss = criterion(outputs, labels)
        loss_val = loss.data.cpu().numpy().reshape(1)[0]
        loss.backward()
        
        optimizer.step()
        
        itern += 1
        if itern%500 ==0:
            correct = 0
            total = 0
            for test_images, test_labels in test_loader:
                test_images = Variable(test_images.cuda())
                test_preds = model(test_images)
                _, predicted = torch.max(test_preds.data, 1)  
                predicted = predicted.cpu()
                total +=  test_labels.size(0)
                correct += (predicted == test_labels).sum()
            correct = correct.numpy().reshape(1)[0]
            accuracy = 100 * correct/total    
            print('Epoch {}, Iter {}, Loss {}, Accuracy {}'.format(epoch, itern,loss_val, accuracy))
print('Training time {}'.format(time.time() - start_time))

6
Epoch 0, Iter 500, Loss 0.0562920942902565, Accuracy 98.06
Epoch 1, Iter 1000, Loss 0.012510638684034348, Accuracy 98.54
Epoch 2, Iter 1500, Loss 0.045518260449171066, Accuracy 98.84
Epoch 3, Iter 2000, Loss 0.11663320660591125, Accuracy 98.87
Epoch 4, Iter 2500, Loss 0.02860969863831997, Accuracy 98.84
Epoch 4, Iter 3000, Loss 0.17760615050792694, Accuracy 98.97
Training time 53.44499611854553
